In [1]:
import numpy as np

def process_obj_with_colors(input_obj_path, output_obj_path):
    """
    Lit un fichier OBJ avec des sommets (et potentiellement des couleurs),
    centre l'objet et le sauvegarde dans un nouveau fichier OBJ
    en préservant les informations de couleur.

    Args:
        input_obj_path (str): Chemin du fichier OBJ d'entrée.
        output_obj_path (str): Chemin du fichier OBJ de sortie.
    """
    vertices = []
    vertex_colors = []
    faces = []
    normals = []
    texcoords = []

    print(f"Lecture du fichier OBJ : {input_obj_path}")
    with open(input_obj_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if not parts:
                continue

            if parts[0] == 'v':
                # Les sommets peuvent avoir 3 (x,y,z) ou 6 (x,y,z,r,g,b) valeurs
                v = [float(p) for p in parts[1:4]]
                vertices.append(v)
                if len(parts) == 7:  # Si des couleurs sont présentes
                    c = [float(p) for p in parts[4:7]]
                    vertex_colors.append(c)
                else:
                    vertex_colors.append(None) # Pas de couleur pour ce sommet
            elif parts[0] == 'vn':
                normals.append([float(p) for p in parts[1:4]])
            elif parts[0] == 'vt':
                texcoords.append([float(p) for p in parts[1:3]])
            elif parts[0] == 'f':
                faces.append(parts[1:])

    if not vertices:
        print("Aucun sommet trouvé dans le fichier OBJ. Opération annulée.")
        return

    # Convertir les sommets en tableau numpy pour faciliter les opérations
    vertices_np = np.array(vertices)

    # Calculer le centre de l'objet (centre de la boîte englobante)
    min_coords = np.min(vertices_np, axis=0)
    max_coords = np.max(vertices_np, axis=0)
    center = (min_coords + max_coords) / 2.0

    print(f"Centre de l'objet détecté : {center}")

    # Centrer les sommets
    centered_vertices_np = vertices_np - center

    print(f"Sauvegarde de l'objet centré dans : {output_obj_path}")
    with open(output_obj_path, 'w') as f:
        # Écrire les sommets centrés avec leurs couleurs (si elles existent)
        for i, v_centered in enumerate(centered_vertices_np):
            f.write(f"v {v_centered[0]:.6f} {v_centered[1]:.6f} {v_centered[2]:.6f}")
            if vertex_colors[i] is not None:
                c = vertex_colors[i]
                f.write(f" {c[0]:.6f} {c[1]:.6f} {c[2]:.6f}")
            f.write("\n")

        # Écrire les normales
        for vn in normals:
            f.write(f"vn {vn[0]:.6f} {vn[1]:.6f} {vn[2]:.6f}\n")

        # Écrire les coordonnées de texture
        for vt in texcoords:
            f.write(f"vt {vt[0]:.6f} {vt[1]:.6f}\n")

        # Écrire les faces
        for face in faces:
            f.write("f " + " ".join(face) + "\n")

    print("Traitement terminé avec succès.")

In [ ]:
input_file = "../graphics/visapp/3d/gorgoile/sommets_visibles_pose_i7-j2.obj"
output_file = "output_centered.obj"

process_obj_with_colors(input_file, output_file)

# Vous pouvez inspecter 'output_centered.obj' pour voir les sommets centrés
# et les couleurs préservées.